<a href="https://colab.research.google.com/github/sunnysaxena/lstm-stock/blob/main/LSTM_plus_Attn_Adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow -qqq
# !pip install keras -qqq
# !pip install yfinance -qqq

In [2]:
import tensorflow as tf
import keras
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Check TensorFlow version
print("TensorFlow Version: ", tf.__version__)


TensorFlow Version:  2.15.0


In [3]:
# Fetch AAPL data
# aapl_data = yf.download('AAPL', start='2020-01-01', end='2024-01-01')

aapl_data = pd.read_csv('https://raw.githubusercontent.com/sunnysaxena/lstm-stock/main/ongc.csv')

# Display the first few rows of the dataframe
aapl_data.head()

,Date,Open,High,Low,Close,Volume
0,2020-01-01,128.75,128.90,126.85,127.45,0
1,2020-01-02,127.65,128.65,127.10,128.05,0
2,2020-01-03,131.00,133.40,128.05,128.45,0
3,2020-01-06,129.70,129.80,125.10,126.25,0
4,2020-01-07,125.60,127.70,125.40,125.75,0


In [4]:
# Checking for missing values
aapl_data.isnull().sum()

# Filling missing values, if any
aapl_data.fillna(method='ffill', inplace=True)

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
aapl_data_scaled = scaler.fit_transform(aapl_data['Close'].values.reshape(-1,1))


In [6]:
X = []
y = []

for i in range(60, len(aapl_data_scaled)):
    X.append(aapl_data_scaled[i-60:i, 0])
    y.append(aapl_data_scaled[i, 0])

In [7]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [8]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [9]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout

# # Model initialization
# model = Sequential()

# # Adding the first LSTM layer
# model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))

# # Adding additional LSTM layers
# model.add(LSTM(units=50, return_sequences=True))
# model.add(LSTM(units=50))


In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()

# Adding LSTM layers
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50, return_sequences=False))  # Only the last time step

# Adding a Dense layer to match the output shape with y_train
model.add(Dense(1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
24/24 [==============================] - 14s 111ms/step - loss: 0.0365 - val_loss: 0.0032
Epoch 2/100
24/24 [==============================] - 1s 23ms/step - loss: 0.0042 - val_loss: 9.2390e-04
Epoch 3/100
24/24 [==============================] - 1s 36ms/step - loss: 0.0032 - val_loss: 8.0361e-04
Epoch 4/100
24/24 [==============================] - 1s 32ms/step - loss: 0.0031 - val_loss: 7.7579e-04
Epoch 5/100
24/24 [==============================] - 1s 36ms/step - loss: 0.0029 - val_loss: 7.4448e-04
Epoch 6/100
24/24 [==============================] - 1s 23ms/step - loss: 0.0028 - val_loss: 7.1939e-04
Epoch 7/100
24/24 [==============================] - 1s 24ms/step - loss: 0.0029 - val_loss: 0.0018
Epoch 8/100
24/24 [==============================] - 1s 29ms/step - loss: 0.0026 - val_loss: 0.0011
Epoch 9/100
24/24 [==============================] - 1s 26ms/step - loss: 0.0024 - val_loss: 9.5393e-04
Epoch 10/100
24/24 [==============================] - 1s 23ms/step - loss:

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, AdditiveAttention, Permute, Reshape, Multiply

model = Sequential()

# Adding LSTM layers with return_sequences=True
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))

# Adding self-attention mechanism
# The attention mechanism
attention = AdditiveAttention(name='attention_weight')
# Permute and reshape for compatibility
model.add(Permute((2, 1)))
model.add(Reshape((-1, X_train.shape[1])))
attention_result = attention([model.output, model.output])
multiply_layer = Multiply()([model.output, attention_result])
# Return to original shape
model.add(Permute((2, 1)))
model.add(Reshape((-1, 50)))

# Adding a Flatten layer before the final Dense layer
model.add(tf.keras.layers.Flatten())

# Final Dense layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
24/24 [==============================] - 4s 40ms/step - loss: 0.0279 - val_loss: 0.0014
Epoch 2/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0072 - val_loss: 0.0022
Epoch 3/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0054 - val_loss: 0.0012
Epoch 4/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0042 - val_loss: 0.0014
Epoch 5/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 6/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0040 - val_loss: 9.2477e-04
Epoch 7/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0035 - val_loss: 9.0961e-04
Epoch 8/100
24/24 [==============================] - 0s 9ms/step - loss: 0.0031 - val_loss: 8.6688e-04
Epoch 9/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0030 - val_loss: 8.4263e-04
Epoch 10/100
24/24 [==============================] - 0s 13ms/step - loss: 0.0024 - val_l

In [12]:
from keras.layers import BatchNormalization

# Adding Dropout and Batch Normalization
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [13]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
# Assume 'data' is your preprocessed dataset
train_size = int(len(aapl_data) * 0.8)
train_data, test_data = aapl_data[:train_size], aapl_data[train_size:]

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 50)            10400     
                                                                 
 lstm_4 (LSTM)               (None, 60, 50)            20200     
                                                                 
 permute (Permute)           (None, 50, 60)            0         
                                                                 
 reshape (Reshape)           (None, 50, 60)            0         
                                                                 
 permute_1 (Permute)         (None, 60, 50)            0         
                                                                 
 reshape_1 (Reshape)         (None, 60, 50)            0         
                                                                 
 flatten (Flatten)           (None, 3000)             

In [16]:
# Assuming X_train and y_train are already defined and preprocessed
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2)

Epoch 1/100
24/24 [==============================] - 8s 42ms/step - loss: 0.7172 - val_loss: 0.3110
Epoch 2/100
24/24 [==============================] - 0s 10ms/step - loss: 0.3985 - val_loss: 0.2544
Epoch 3/100
24/24 [==============================] - 0s 10ms/step - loss: 0.2173 - val_loss: 0.2322
Epoch 4/100
24/24 [==============================] - 0s 10ms/step - loss: 0.1426 - val_loss: 0.2139
Epoch 5/100
24/24 [==============================] - 0s 13ms/step - loss: 0.1138 - val_loss: 0.1970
Epoch 6/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0909 - val_loss: 0.1743
Epoch 7/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0809 - val_loss: 0.1703
Epoch 8/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0739 - val_loss: 0.1387
Epoch 9/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0600 - val_loss: 0.1205
Epoch 10/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0540 - val_loss: 0.1243

In [17]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
24/24 [==============================] - 0s 16ms/step - loss: 0.0128 - val_loss: 0.0056
Epoch 2/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0146 - val_loss: 4.5751e-04
Epoch 3/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0049
Epoch 4/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0131 - val_loss: 3.1080e-04
Epoch 5/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0123 - val_loss: 5.7009e-04
Epoch 6/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0057
Epoch 7/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0119 - val_loss: 0.0027
Epoch 8/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0114 - val_loss: 0.0034
Epoch 9/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0130 - val_loss: 4.8539e-04
Epoch 10/100
24/24 [==============================] - 0s 10ms/step - loss: 0.0128 - 

In [18]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard, CSVLogger

# Callback to save the model periodically
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# Callback to reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

# Callback for TensorBoard
tensorboard = TensorBoard(log_dir='./logs')

# Callback to log details to a CSV file
csv_logger = CSVLogger('training_log.csv')

# Combining all callbacks
callbacks_list = [early_stopping, model_checkpoint, reduce_lr, tensorboard, csv_logger]

# Fit the model with the callbacks
history = model.fit(X_train, y_train, epochs=100, batch_size=25, validation_split=0.2, callbacks=callbacks_list)

Epoch 1/100
24/24 [==============================] - 0s 15ms/step - loss: 0.0135 - val_loss: 9.2769e-04 - lr: 0.0010
Epoch 2/100
13/24 [===============>..............] - ETA: 0s - loss: 0.0130

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


24/24 [==============================] - 0s 11ms/step - loss: 0.0131 - val_loss: 0.0112 - lr: 0.0010
Epoch 3/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0131 - val_loss: 0.0021 - lr: 0.0010
Epoch 4/100
24/24 [==============================] - 0s 14ms/step - loss: 0.0131 - val_loss: 8.4342e-04 - lr: 0.0010
Epoch 5/100
24/24 [==============================] - 0s 13ms/step - loss: 0.0124 - val_loss: 0.0018 - lr: 0.0010
Epoch 6/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0125 - val_loss: 0.0013 - lr: 0.0010
Epoch 7/100
24/24 [==============================] - 0s 11ms/step - loss: 0.0120 - val_loss: 0.0016 - lr: 1.0000e-04
Epoch 8/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0132 - val_loss: 0.0010 - lr: 1.0000e-04
Epoch 9/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0128 - val_loss: 0.0015 - lr: 1.0000e-04
Epoch 10/100
24/24 [==============================] - 0s 12ms/step - loss: 0.0110 - val_loss

In [19]:
# Convert X_test and y_test to Numpy arrays if they are not already
X_test = np.array(X_test)
y_test = np.array(y_test)

# Ensure X_test is reshaped similarly to how X_train was reshaped
# This depends on how you preprocessed the training data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Now evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print("Test Loss: ", test_loss)

6/6 [==============================] - 1s 8ms/step - loss: 0.0086
Test Loss:  0.008600901812314987


In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Making predictions
y_pred = model.predict(X_test)

# Calculating MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Mean Absolute Error: ", mae)
print("Root Mean Square Error: ", rmse)

6/6 [==============================] - 1s 5ms/step
Mean Absolute Error:  0.08798147605706495
Root Mean Square Error:  0.09274104856821185


In [21]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetching the latest 60 days of AAPL stock data
# data = yf.download('AAPL', period='60d', interval='1d')

data = pd.read_csv('https://raw.githubusercontent.com/sunnysaxena/lstm-stock/main/ongc_test.csv')

# Selecting the 'Close' price and converting to numpy array
closing_prices = data['Close'].values

# Scaling the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(closing_prices.reshape(-1,1))

# Since we need the last 60 days to predict the next day, we reshape the data accordingly
X_latest = np.array([scaled_data[-60:].reshape(60)])

# Reshaping the data for the model (adding batch dimension)
X_latest = np.reshape(X_latest, (X_latest.shape[0], X_latest.shape[1], 1))

# Making predictions for the next 4 candles
predicted_stock_price = model.predict(X_latest)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

print("Predicted Stock Prices for the next 4 days: ", predicted_stock_price)


1/1 [==============================] - 0s 25ms/step
Predicted Stock Prices for the next 4 days:  [[270.7693]]


In [22]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Fetch the latest 60 days of AAPL stock data
# data = yf.download('AAPL', period='60d', interval='1d')

data = pd.read_csv('https://raw.githubusercontent.com/sunnysaxena/lstm-stock/main/ongc_test.csv')

# Select 'Close' price and scale it
closing_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

# Predict the next 4 days iteratively
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)  # Most recent 60 days

for i in range(4):  # Predicting 4 days
    # Get the prediction (next day)
    next_prediction = model.predict(current_batch)

    # Reshape the prediction to fit the batch dimension
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)

    # Append the prediction to the batch used for predicting
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)

    # Inverse transform the prediction to the original price scale
    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

print("Predicted Stock Prices for the next 4 days: ", predicted_prices)

1/1 [==============================] - 0s 19ms/step
Predicted Stock Prices for the next 4 days:  [270.7693, 261.76144, 255.06328, 251.33768]


In [23]:
# !pip install mplfinance -qqq

In [25]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

# Assuming 'data' is your DataFrame with the fetched AAPL stock data
# Make sure it contains Open, High, Low, Close, and Volume columns

# Creating a list of dates for the predictions
last_date = data.index[-1]

In [26]:
next_day = last_date + pd.Timedelta(days=1)
prediction_dates = pd.date_range(start=next_day, periods=4)

# Assuming 'predicted_prices' is your list of predicted prices for the next 4 days
predictions_df = pd.DataFrame(index=prediction_dates, data=predicted_prices, columns=['Close'])

# Plotting the actual data with mplfinance
mpf.plot(data, type='candle', style='charles', volume=True)

# Overlaying the predicted data
plt.figure(figsize=(18,6))
plt.plot(predictions_df.index, predictions_df['Close'], linestyle='dashed', marker='o', color='red')

plt.title("AAPL Stock Price with Predicted Next 4 Days")
plt.show()

TypeError: unsupported operand type(s) for +: 'int' and 'Timedelta'

In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

# Fetch the latest 60 days of AAPL stock data
data = yf.download('AAPL', period='64d', interval='1d') # Fetch 64 days to display last 60 days in the chart

# Select 'Close' price and scale it
closing_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

# Predict the next 4 days iteratively
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)  # Most recent 60 days

for i in range(4):  # Predicting 4 days
    next_prediction = model.predict(current_batch)
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

# Creating a list of dates for the predictions
last_date = data.index[-1]
next_day = last_date + pd.Timedelta(days=1)
prediction_dates = pd.date_range(start=next_day, periods=4)

# Adding predictions to the DataFrame
predicted_data = pd.DataFrame(index=prediction_dates, data=predicted_prices, columns=['Close'])

# Combining both actual and predicted data
combined_data = pd.concat([data['Close'], predicted_data['Close']])
combined_data = combined_data[-64:] # Last 60 days of actual data + 4 days of predictions

# Plotting the data
plt.figure(figsize=(18,6))
plt.plot(combined_data, linestyle='-', marker='o', color='blue')
plt.title("AAPL Stock Price: Last 60 Days and Next 4 Days Predicted")
plt.show()


In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt

# Fetch the latest 60 days of AAPL stock data
data = yf.download('AAPL', period='64d', interval='1d') # Fetch 64 days to display last 60 days in the chart

# Select 'Close' price and scale it
closing_prices = data['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

# Predict the next 4 days iteratively
predicted_prices = []
current_batch = scaled_data[-60:].reshape(1, 60, 1)  # Most recent 60 days

for i in range(4):  # Predicting 4 days
    next_prediction = model.predict(current_batch)
    next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
    current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
    predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

# Creating a list of dates for the predictions
last_date = data.index[-1]
next_day = last_date + pd.Timedelta(days=1)
prediction_dates = pd.date_range(start=next_day, periods=4)

# Adding predictions to the DataFrame
predicted_data = pd.DataFrame(index=prediction_dates, data=predicted_prices, columns=['Close'])

# Combining both actual and predicted data
combined_data = pd.concat([data['Close'], predicted_data['Close']])
combined_data = combined_data[-64:] # Last 60 days of actual data + 4 days of predictions

# Plotting the actual data
plt.figure(figsize=(18,6))
plt.plot(data.index[-60:], data['Close'][-60:], linestyle='-', marker='o', color='blue', label='Actual Data')

# Plotting the predicted data
plt.plot(prediction_dates, predicted_prices, linestyle='-', marker='o', color='red', label='Predicted Data')

plt.title("AAPL Stock Price: Last 60 Days and Next 4 Days Predicted")
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [27]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta


def predict_stock_price(input_date):
    # Check if the input date is a valid date format
    try:
        input_date = pd.to_datetime(input_date)
    except ValueError:
        print("Invalid Date Format. Please enter date in YYYY-MM-DD format.")
        return

    # Fetch data from yfinance
    end_date = input_date
    start_date = input_date - timedelta(days=90)  # Fetch more days to ensure we have 60 trading days
    data = yf.download('AAPL', start=start_date, end=end_date)

    if len(data) < 60:
        print("Not enough historical data to make a prediction. Try an earlier date.")
        return

    # Prepare the data
    closing_prices = data['Close'].values[-60:]  # Last 60 days
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(closing_prices.reshape(-1, 1))

    # Make predictions
    predicted_prices = []
    current_batch = scaled_data.reshape(1, 60, 1)

    for i in range(4):  # Predicting 4 days
        next_prediction = model.predict(current_batch)
        next_prediction_reshaped = next_prediction.reshape(1, 1, 1)
        current_batch = np.append(current_batch[:, 1:, :], next_prediction_reshaped, axis=1)
        predicted_prices.append(scaler.inverse_transform(next_prediction)[0, 0])

    # Output the predictions
    for i, price in enumerate(predicted_prices, 1):
        print(f"Day {i} prediction: {price}")

# Example use
user_input = input("Enter a date (YYYY-MM-DD) to predict AAPL stock for the next 4 days: ")
predict_stock_price(user_input)

Enter a date (YYYY-MM-DD) to predict AAPL stock for the next 4 days: 2024-04-29


[*********************100%%**********************]  1 of 1 completed

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 19ms/step
Day 1 prediction: 169.30319213867188
Day 2 prediction: 170.191650390625
Day 3 prediction: 171.6841583251953
Day 4 prediction: 173.26043701171875


In [ ]:
aapl_data.head(1)

In [ ]:
aapl_data.tail(1)

In [ ]:
data.head(1)

In [ ]:
data.tail(1)